In [1]:
from dotenv import load_dotenv
import json

load_dotenv()

True

# Tool Schema Tests

In [ ]:
from typing import Any

from anthropic_agent.tools import ToolRegistry, tool


@tool
def get_weather(location: str, unit: str = "celsius") -> str:
    """Get the current weather in a given location.
    
    This function retrieves weather information for the specified location
    and returns it in the requested temperature unit.
    
    Args:
        location: The city and state, e.g. "San Francisco, CA"
        unit: The unit of temperature, either "celsius" or "fahrenheit"
    
    Returns:
        String describing the current weather
    """
    # Mock implementation
    return f"The weather in {location} is 22°{unit[0].upper()}"

@tool
def plan_travel_itinerary(
    travelers: list[dict[str, str]],
    destination: str,
    start_date: str,
    end_date: str,
    preferences: dict[str, Any] | None = None,
) -> str:
    """Plan a multi-day travel itinerary.
    
    This tool accepts structured traveler information along with
    trip preferences and returns a concise summary. It showcases
    nested objects and optional parameters in the schema.
    
    Args:
        travelers: List of traveler profiles including name and role
        destination: Target city or country for the itinerary
        start_date: Trip start in ISO format (YYYY-MM-DD)
        end_date: Trip end in ISO format (YYYY-MM-DD)
        preferences: Optional mapping of preference category to value
    
    Returns:
        String summary describing the generated itinerary
    """
    traveler_list = ", ".join(
        f"{traveler['name']} ({traveler.get('role', 'guest')})"
        for traveler in travelers
    )
    pref_summary = (
        ", ".join(f"{key}={value}" for key, value in preferences.items())
        if preferences
        else "standard preferences"
    )
    return (
        f"Trip to {destination} from {start_date} to {end_date} for "
        f"{len(travelers)} traveler(s): {traveler_list}. Preferences: {pref_summary}."
    )


registry = ToolRegistry()
registry.register_tools([get_weather, plan_travel_itinerary])
print("Registered tools:", [schema["name"] for schema in registry.get_schemas()])

Registered tools: ['get_weather', 'plan_travel_itinerary']


## Export as OpenAI Format

In [3]:
import json

openai_tool_payload = registry.get_schemas(schema_type="openai")

print(json.dumps(openai_tool_payload, indent=2))

[
  {
    "type": "function",
    "function": {
      "name": "get_weather",
      "description": "Get the current weather in a given location.\n\nThis function retrieves weather information for the specified location\nand returns it in the requested temperature unit.",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. \"San Francisco, CA\""
          },
          "unit": {
            "type": "string",
            "nullable": true,
            "description": "The unit of temperature, either \"celsius\" or \"fahrenheit\""
          }
        },
        "required": [
          "location"
        ]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "plan_travel_itinerary",
      "description": "Plan a multi-day travel itinerary.\n\nThis tool accepts structured traveler information along with\ntrip preferences and returns a concise su

## Export as Anthropic Format

In [4]:
import json

anthropic_tool_schemas = registry.get_schemas()
print(json.dumps(anthropic_tool_schemas, indent=2))

[
  {
    "name": "get_weather",
    "description": "Get the current weather in a given location.\n\nThis function retrieves weather information for the specified location\nand returns it in the requested temperature unit.",
    "input_schema": {
      "type": "object",
      "properties": {
        "location": {
          "type": "string",
          "description": "The city and state, e.g. \"San Francisco, CA\""
        },
        "unit": {
          "type": "string",
          "nullable": true,
          "description": "The unit of temperature, either \"celsius\" or \"fahrenheit\""
        }
      },
      "required": [
        "location"
      ]
    }
  },
  {
    "name": "plan_travel_itinerary",
    "description": "Plan a multi-day travel itinerary.\n\nThis tool accepts structured traveler information along with\ntrip preferences and returns a concise summary. It showcases\nnested objects and optional parameters in the schema.",
    "input_schema": {
      "type": "object",
      "

# Basic Agent with Tools

In [ ]:
import asyncio

from anthropic_agent.core import AnthropicAgent
from anthropic_agent.tools import SAMPLE_TOOL_FUNCTIONS

agent = AnthropicAgent(
        system_prompt="You are a helpful assistant that should help the user with their questions.",
        model="claude-sonnet-4-5",
        tools=SAMPLE_TOOL_FUNCTIONS,
    )

In [3]:
agent_uuid = agent.agent_uuid
print(agent_uuid)

6ab69301-74de-40df-b8cc-cbc8386da99e


In [4]:
print(agent)
# print(json.dumps(agent.tool_schemas, indent=2))

{
  "agent_uuid": "6ab69301-74de-40df-b8cc-cbc8386da99e",
  "system_prompt": "You are a helpful assistant that should help the user with their questions.",
  "model": "claude-sonnet-4-5",
  "max_steps": 50,
  "thinking_tokens": 0,
  "max_tokens": 2048,
  "stream_meta_history_and_tool_results": false,
  "beta_headers": [],
  "container_id": null,
  "messages_count": 0,
  "conversation_history_count": 0,
  "agent_logs_count": 0,
  "max_retries": 5,
  "base_delay": 1.0,
  "api_kwargs": {},
  "formatter": "xml",
  "compactor": null,
  "memory_store": null,
  "final_answer_check": false,
  "db_backend": "FilesystemBackend",
  "tools": [
    "add",
    "subtract",
    "multiply",
    "divide"
  ]
}


In [5]:
prompt = "What is 5 + (10 * 2)?"

## Running w/o queue

In [6]:
_res_wo_queue = agent.run(
    prompt=prompt,
)
res_wo_queue = await _res_wo_queue

In [7]:
print(res_wo_queue)

{
  "final_message": "ParsedBetaMessage(id='msg_012WphmLkdCcVnJ9eKxs1nKv', container=None, content=[ParsedBetaTextBlock(citations=None, text='The answer is **25**.\\n\\nTo break it down:\\n- 10 * 2 = 20\\n- 5 + 20 = 25', type='text', parsed_output=None)], context_management=None, model='claude-sonnet-4-5-20250929', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=BetaUsage(cache_creation=BetaCacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=1208, output_tokens=39, server_tool_use=None, service_tier='standard'))",
  "conversation_history": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What is 5 + (10 * 2)?"
        }
      ]
    },
    {
      "content": [
        {
          "text": "I need to calculate 5 + (10 * 2) following the order of operations (multiplication first, then addition).\n\nLet me brea

In [8]:
print(res_wo_queue.final_message.content[0].text)

The answer is **25**.

To break it down:
- 10 * 2 = 20
- 5 + 20 = 25


## Running with Queue

### XML Formatter

In [9]:
queue: asyncio.Queue[str | None] = asyncio.Queue()

async def print_queue() -> None:
        while True:
            chunk = await queue.get()
            if chunk is None:
                break
            print(chunk, end="", flush=True)
            queue.task_done()

printer = asyncio.create_task(print_queue())

result = await agent.run(prompt, queue)

await queue.put(None)  # signal printer to stop
await printer

<content-block:text>I need to calculate 5 + (10 * 2) following the order of operations (multiplication first, then addition).

Let me break this down:
1. First: 10 * 2
2. Then: 5 + (result from step 1)
</content-block:text><content-block:tool_call>
{
  "type": "tool_use",
  "id": "toolu_018Szx1wa2LqZwZRTjftT8eJ",
  "name": "multiply",
  "input": {
    "a": 10,
    "b": 2
  }
}
</content-block:tool_call><content-block:tool_call>
{
  "type": "tool_use",
  "id": "toolu_013CYux69yK4ZNSpkjWMdvmf",
  "name": "add",
  "input": {
    "a": 5,
    "b": 20
  }
}
</content-block:tool_call><content-block:text>The answer is **25**.

To break it down:
- 10 * 2 = 20
- 5 + 20 = 25
</content-block:text>

### Raw formatter

In [10]:
queue2: asyncio.Queue[str | None] = asyncio.Queue()

async def print_queue() -> None:
        while True:
            chunk = await queue2.get()
            if chunk is None:
                break
            print(chunk, end="\n", flush=True)
            queue2.task_done()

printer2 = asyncio.create_task(print_queue())

result2 = await agent.run(prompt, queue2, "raw")

await queue2.put(None)  # signal printer to stop
await printer2

{"message":{"id":"msg_01NC5kRQJNXZgXoCdBiVTt6c","container":null,"content":[],"context_management":null,"model":"claude-sonnet-4-5-20250929","role":"assistant","stop_reason":null,"stop_sequence":null,"type":"message","usage":{"cache_creation":{"ephemeral_1h_input_tokens":0,"ephemeral_5m_input_tokens":0},"cache_creation_input_tokens":0,"cache_read_input_tokens":0,"input_tokens":1542,"output_tokens":8,"server_tool_use":null,"service_tier":"standard"}},"type":"message_start"}
{"content_block":{"citations":null,"text":"","type":"text"},"index":0,"type":"content_block_start"}
{"delta":{"text":"I need to calculate 5 +","type":"text_delta"},"index":0,"type":"content_block_delta"}
{"delta":{"text":" (10 * 2) following","type":"text_delta"},"index":0,"type":"content_block_delta"}
{"delta":{"text":" the order of operations (multiplication first,","type":"text_delta"},"index":0,"type":"content_block_delta"}
{"delta":{"text":" then addition).\n\nLet me break this","type":"text_delta"},"index":0,"t

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/.venv/lib/python3.12/site-packages/pydantic/main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `ParsedBetaTextBlock[TypeVar]` - serialized value may not be as expected [field_name='content', input_value=ParsedBetaTextBlock(citat...xt', parsed_output=None), input_type=ParsedBetaTextBlock])
  PydanticSerializationUnexpectedValue(Expected `BetaThinkingBlock` - serialized value may not be as expected [field_name='content', input_value=ParsedBetaTextBlock(citat...xt', parsed_output=None), input_type=ParsedBetaTextBlock])
  PydanticSerializationUnexpectedValue(Expected `BetaRedactedThinkingBlock` - serialized value may not be as expected [field_name='content', input_value=ParsedBetaTextBlock(citat...xt', parsed_output=None), input_type=ParsedBetaTextBlock])
  PydanticSerializationUnexpectedValue(Expected `BetaToolUseBlock` - serialized value may not be as expected [field_name='conten

{"message":{"id":"msg_01PqStfDrrHzRygs2joMLJQR","container":null,"content":[],"context_management":null,"model":"claude-sonnet-4-5-20250929","role":"assistant","stop_reason":null,"stop_sequence":null,"type":"message","usage":{"cache_creation":{"ephemeral_1h_input_tokens":0,"ephemeral_5m_input_tokens":0},"cache_creation_input_tokens":0,"cache_read_input_tokens":0,"input_tokens":1685,"output_tokens":24,"server_tool_use":null,"service_tier":"standard"}},"type":"message_start"}
{"content_block":{"id":"toolu_0143nRmePSm4TGDGzsAwgvhE","input":{},"name":"add","type":"tool_use","caller":null},"index":0,"type":"content_block_start"}
{"delta":{"partial_json":"","type":"input_json_delta"},"index":0,"type":"content_block_delta"}
{"delta":{"partial_json":"{\"a\": 5","type":"input_json_delta"},"index":0,"type":"content_block_delta"}
{"delta":{"partial_json":", \"","type":"input_json_delta"},"index":0,"type":"content_block_delta"}
{"delta":{"partial_json":"b\": 20}","type":"input_json_delta"},"index"

In [11]:
result2.__dict__

{'final_message': ParsedBetaMessage(id='msg_01DTd2hxPKAcTD2PrDpErBq1', container=None, content=[ParsedBetaTextBlock(citations=None, text='The answer is **25**.\n\nTo break it down:\n- 10 * 2 = 20\n- 5 + 20 = 25', type='text', parsed_output=None)], context_management=None, model='claude-sonnet-4-5-20250929', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=BetaUsage(cache_creation=BetaCacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=1766, output_tokens=39, server_tool_use=None, service_tier='standard')),
 'conversation_history': [{'role': 'user',
   'content': [{'type': 'text', 'text': 'What is 5 + (10 * 2)?'}]},
  {'content': [{'text': 'I need to calculate 5 + (10 * 2) following the order of operations (multiplication first, then addition).\n\nLet me break this down:\n1. First: 10 * 2\n2. Then: 5 + (result from step 1)',
     'type': 'text'},
    {'id': 'too

In [13]:
print(result2.final_message.content[0].text)

The answer is **25**.

To break it down:
- 10 * 2 = 20
- 5 + 20 = 25


# File Generation Agent

In [30]:
import asyncio

from anthropic_agent import AnthropicAgent
from anthropic_agent.src.tools import SAMPLE_TOOL_FUNCTIONS

agent = AnthropicAgent(
        system_prompt="You are a helpful assistant that should help the user with their questions.",
        model="claude-sonnet-4-5",
        thinking_tokens=1024,
        max_tokens=64000,
        server_tools=[{
            "type": "code_execution_20250825",
            "name": "code_execution"
        }],
        beta_headers=["files-api-2025-04-14", "code-execution-2025-08-25"],
        file_backend="local"
        )

In [31]:
agent_uuid = agent.agent_uuid
print(agent_uuid)

26b31c43-40e7-407f-a1ce-e5a5f5e4e0aa


In [32]:
print(agent)

{
  "agent_uuid": "26b31c43-40e7-407f-a1ce-e5a5f5e4e0aa",
  "system_prompt": "You are a helpful assistant that should help the user with their questions.",
  "model": "claude-sonnet-4-5",
  "max_steps": 50,
  "thinking_tokens": 1024,
  "max_tokens": 64000,
  "stream_meta_history_and_tool_results": false,
  "beta_headers": [
    "files-api-2025-04-14",
    "code-execution-2025-08-25"
  ],
  "container_id": null,
  "messages_count": 0,
  "conversation_history_count": 0,
  "agent_logs_count": 0,
  "max_retries": 5,
  "base_delay": 1.0,
  "formatter": "xml",
  "compactor": null,
  "memory_store": null,
  "final_answer_check": false,
  "db_backend": "FilesystemBackend",
  "tools": []
}


In [33]:
prompt = {
    "role": "user",
    "content": [
        {
            "type": "text",
            "text": "Generate a sample invoice image for a hypothetical restuarant and return that image."
        },
    ]
}

In [34]:
queue: asyncio.Queue[str | None] = asyncio.Queue()

async def print_queue() -> None:
        while True:
            chunk = await queue.get()
            if chunk is None:
                break
            print(chunk, end="", flush=True)
            queue.task_done()

printer = asyncio.create_task(print_queue())

result = await agent.run(prompt, queue, "xml")

await queue.put(None)  # signal printer to stop
await printer

DEBUG:anthropic_agent.src.core.agent:Anthropic count_tokens params: {'model': 'claude-sonnet-4-5', 'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': 'Generate a sample invoice image for a hypothetical restuarant and return that image.'}]}], 'system': 'You are a helpful assistant that should help the user with their questions.', 'thinking': {'type': 'enabled', 'budget_tokens': 1024}}
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages/count_tokens "HTTP/1.1 200 OK"
INFO:anthropic_agent.src.core.retry:Attempting Anthropic stream call (attempt 1/5)
DEBUG:anthropic_agent.src.core.retry:Anthropic request params: {'model': 'claude-sonnet-4-5', 'max_tokens': 64000, 'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': 'Generate a sample invoice image for a hypothetical restuarant and return that image.'}]}], 'system': 'You are a helpful assistant that should help the user with their questions.', 'thinking': {'type': 'enabled', 'budget_tokens': 102

<content-block:thinking>The user wants me to generate a sample invoice image for a hypothetical restaurant. I'll need to:
1. Create the invoice content (probably as HTML or using a Python library to generate an image)
2. Convert it to an image format
3. Save it to OUTPUT_DIR so it gets exported to the user

I think the best approach is to use Python with PIL (Pillow) to create an invoice image, or I could create an HTML file and convert it to an image using a tool like wkhtmltoimage if available. Let me check what's available in the environment - the description mentions imagemagick is available, and Python libraries for PDF manipulation.

I'll create a Python script that uses PIL/Pillow to draw a restaurant invoice with typical invoice elements like:
- Restaurant name and logo/header
- Invoice number and date
- Table number
- Items ordered with prices
- Subtotal, tax, and total
- Payment information

Let me create this invoice image.
</content-block:thinking><content-block:text>I'll c

INFO:anthropic_agent.src.core.retry:Anthropic stream completed successfully (stop_reason: end_turn)
DEBUG:anthropic_agent.src.core.agent:Assistant message: {'content': [{'signature': 'Er0ICkYIChgCKkDrj2nDU4Vvjr4Qbpeg02YbjaV/Rmi2L7osIQAtVS+egRgJRYdZf0mqBpcW6tyPToSJ5WoUq8DeqpKYfCnwUB2SEgwk3Gg8E18AZ7pAZcQaDJj7xS1mYkm2OXgJoyIwXds9SVwP6kk0NPi3Ttt7CbNFgbaJm/OoJb0IA2JaPx66EXgMJB+p4PIv8a553RB/KqQHcvubL8NOjKFzcyyP4Yvm/6CBmREN6lzqYN3+Jo1/584txf6ZkNbPG6wU5kE8okV+iCtv5+dp176VC71aX051XT3Q2P+svDSByJD7Izi9bjv1Qd+9zPVr4v4Cl9rARN/B4Eu8VE8E+bD/vRhsO6F+bZhf65I+7d4MNgIfnk99hQloQvV/3yp18GNcmrF7MDEKd/WBPrdqsdRdu+Jk0J3Xvm/pjOWceYOghDI/MXhUwihcuaUQvvZhqMVPUTQQtrVoCVtGziw/AZRf2+6FSCMRypN4mWMLup4i+MdhQzPDWkTGZb1YP6ywjPKNLMCa6982Zq479KdPODeURzzlEHLb0HrwzIQla6aeauXin8RVoAxJGjxcYKOExP+r2YfFa8sQp7H2sxNC+dTWeToexI+yUgtvYzAwAhkaxTZZA8dbuY4YtACKetwQECZuQ6luRCe/t06jYO6ydjeiVS8ARMAu+xwY+Vo2acTOpGhSVj2FviXf6yIKSnsuWl1730Zr6BNUBv3zUXRjQez/W9P5VaWDkvxKsTyddxTJT79eHECrDIISq4jL8q1n96XufjOyu/PQpj2ant5GlYtDNVheBJBraFq/dIPhTJkQ


<content-block:meta_files>
{
  "file_ids": [
    "file_011CVYNK4jUL6zUiEfiJdcgH"
  ]
}
</content-block:meta_files>

In [35]:
result.__dict__

{'final_message': ParsedBetaMessage(id='msg_01PYgBZfUc6XCeH2JsVSkuMF', container=BetaContainer(id='container_011CVYNF8uskvWm6d8Xg8JaL', expires_at=datetime.datetime(2025, 11, 27, 13, 8, 42, 800457, tzinfo=TzInfo(0)), skills=None), content=[BetaThinkingBlock(signature='Er0ICkYIChgCKkDrj2nDU4Vvjr4Qbpeg02YbjaV/Rmi2L7osIQAtVS+egRgJRYdZf0mqBpcW6tyPToSJ5WoUq8DeqpKYfCnwUB2SEgwk3Gg8E18AZ7pAZcQaDJj7xS1mYkm2OXgJoyIwXds9SVwP6kk0NPi3Ttt7CbNFgbaJm/OoJb0IA2JaPx66EXgMJB+p4PIv8a553RB/KqQHcvubL8NOjKFzcyyP4Yvm/6CBmREN6lzqYN3+Jo1/584txf6ZkNbPG6wU5kE8okV+iCtv5+dp176VC71aX051XT3Q2P+svDSByJD7Izi9bjv1Qd+9zPVr4v4Cl9rARN/B4Eu8VE8E+bD/vRhsO6F+bZhf65I+7d4MNgIfnk99hQloQvV/3yp18GNcmrF7MDEKd/WBPrdqsdRdu+Jk0J3Xvm/pjOWceYOghDI/MXhUwihcuaUQvvZhqMVPUTQQtrVoCVtGziw/AZRf2+6FSCMRypN4mWMLup4i+MdhQzPDWkTGZb1YP6ywjPKNLMCa6982Zq479KdPODeURzzlEHLb0HrwzIQla6aeauXin8RVoAxJGjxcYKOExP+r2YfFa8sQp7H2sxNC+dTWeToexI+yUgtvYzAwAhkaxTZZA8dbuY4YtACKetwQECZuQ6luRCe/t06jYO6ydjeiVS8ARMAu+xwY+Vo2acTOpGhSVj2FviXf6yIKSnsuWl1730Zr6BNUBv3zUXRjQez

# Agent with Image/File + Thinking + Tool Calls (Server + Client)

In [2]:
test_image = "./currency_receipt_usd_jpy.png"
# Convert image to base64
import base64

with open(test_image, "rb") as image_file:
    base64_image = base64.b64encode(image_file.read()).decode('utf-8')

In [3]:
# Defining the tools
from anthropic_agent.src.tools import tool

@tool
def get_rate(source: str, target: str) -> str:
    '''
    Get the exchange rate from source to target currency.
    Args:
        source: The source currency code. It can be INR, USD, or JPY.
        target: The target currency code. It can be INR, USD, or JPY.
    Returns:
        A string describing the exchange rate from source to target currency
    '''
    # For testing: return fixed mock rates
    mock_rates = {
        ("INR", "USD"): 0.012,
        ("USD", "INR"): 83.1,
        ("INR", "JPY"): 1.5,
        ("JPY", "INR"): 66.7,
        ("USD", "JPY"): 110.0,
        ("JPY", "USD"): 0.0091,
    }
    return f"The exchange rate from {source} to {target} is {mock_rates[(source, target)]}"

In [12]:
import asyncio

from anthropic_agent import AnthropicAgent
from anthropic_agent.src.tools import SAMPLE_TOOL_FUNCTIONS

agent = AnthropicAgent(
        system_prompt="You are a helpful assistant that should help the user with their questions.",
        model="claude-sonnet-4-5",
        thinking_tokens=1024,
        max_tokens=64000,
        tools=SAMPLE_TOOL_FUNCTIONS + [get_rate],
        server_tools=[{
            "type": "code_execution_20250825",
            "name": "code_execution"
        }],
        beta_headers=["files-api-2025-04-14", "code-execution-2025-08-25"],
        file_backend="local"
        )

In [13]:
agent_uuid = agent.agent_uuid
print(agent_uuid)

8957fcff-ebb0-4267-b448-d375d8e184f4


In [14]:
print(agent)

{
  "agent_uuid": "8957fcff-ebb0-4267-b448-d375d8e184f4",
  "system_prompt": "You are a helpful assistant that should help the user with their questions.",
  "model": "claude-sonnet-4-5",
  "max_steps": 50,
  "thinking_tokens": 1024,
  "max_tokens": 64000,
  "stream_meta_history_and_tool_results": false,
  "beta_headers": [
    "files-api-2025-04-14",
    "code-execution-2025-08-25"
  ],
  "container_id": null,
  "messages_count": 0,
  "conversation_history_count": 0,
  "agent_logs_count": 0,
  "max_retries": 5,
  "base_delay": 1.0,
  "formatter": "xml",
  "compactor": null,
  "memory_store": null,
  "final_answer_check": false,
  "db_backend": "FilesystemBackend",
  "tools": [
    "add",
    "subtract",
    "multiply",
    "divide",
    "get_rate"
  ]
}


In [15]:
prompt_1 = {
    "role": "user",
    "content": [
        {
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/png",
                "data": base64_image,
            },
        },
        {
            "type": "text",
            "text": "I paid the receipt in USD. What is the amount in INR?"
        },
    ]
}

In [16]:
import logging

logging.basicConfig(level=logging.INFO)  # or configure handlers as you prefer
logging.getLogger("anthropic_agent.src.core.agent").setLevel(logging.DEBUG)
logging.getLogger("anthropic_agent.src.core.retry").setLevel(logging.DEBUG)


In [17]:
queue: asyncio.Queue[str | None] = asyncio.Queue()

async def print_queue() -> None:
        while True:
            chunk = await queue.get()
            if chunk is None:
                break
            print(chunk, end="", flush=True)
            queue.task_done()

printer = asyncio.create_task(print_queue())

result = await agent.run(prompt_1, queue, "xml")

await queue.put(None)  # signal printer to stop
await printer

DEBUG:anthropic_agent.src.core.agent:Anthropic count_tokens params: {'model': 'claude-sonnet-4-5', 'messages': [{'role': 'user', 'content': [{'type': 'image', 'source': {'type': 'base64', 'media_type': 'image/png', 'data': '...'}}, {'type': 'text', 'text': 'I paid the receipt in USD. What is the amount in INR?'}]}], 'system': 'You are a helpful assistant that should help the user with their questions.', 'tools': [{'name': 'add', 'description': 'Add two numbers together and return the sum.\n\nThis function takes two numbers and returns their sum as a string.', 'input_schema': {'type': 'object', 'properties': {'a': {'type': 'number', 'description': 'The first number to add'}, 'b': {'type': 'number', 'description': 'The second number to add'}}, 'required': ['a', 'b']}}, {'name': 'subtract', 'description': 'Subtract the second number from the first number.\n\nThis function subtracts b from a and returns the result as a string.', 'input_schema': {'type': 'object', 'properties': {'a': {'type

<content-block:thinking>The user paid in USD and wants to know the amount in INR (Indian Rupees). The total in USD is $38.44.

I need to:
1. Get the exchange rate from USD to INR
2. Convert $38.44 to INR

Let me use the get_rate function to get the USD to INR exchange rate, then multiply to get the final amount.
</content-block:thinking><content-block:text>I'll help you convert the USD amount to INR. Let me get the current exchange rate and calculate the amount.
</content-block:text><content-block:tool_call>
{
  "type": "tool_use",
  "id": "toolu_01B2n6LUreCFPaU9p44L9bvS",
  "name": "get_rate",
  "input": {
    "source": "USD",
    "target": "INR"
  }
}
</content-block:tool_call>

INFO:anthropic_agent.src.core.retry:Anthropic stream completed successfully (stop_reason: tool_use)
DEBUG:anthropic_agent.src.core.agent:Assistant message: {'content': [{'signature': 'EsQDCkYIChgCKkBeW1i5nXw+HOoIF7P/7KpNrly1Jyi6/dlo6xhKqwUcYBiT3smd3tnd6AmUXWr+8M4QNiDvBhK06M9REvh/79dfEgwRNMyCFIAO5lGLOBgaDO+zLwEMqo+b+nnFISIwLAAEPaJooG+ncv+KOls1ig3enE3su57+CKnstqMDozZaOUMyNTKcQ2BpQ3wbzT2gKqsCJwdKqcJ0qlqO5LzR3gHCYzUSlQFANzhm7FQuCY4/FJJmxdMRqEIBii9nWs8izRAuqyAyJspGBN3BLw/Ul0wiJPNuk2+sNOg6iQ1ZEoxhJk/uDlDr1zCLtDjeHNRYNIMtiLe29zM7cy2VprNQsevrJMcB8hsFrImeOK70smzFUUM0wfw0/Tn854UUiC7ewsfExnCDKHDwe+tVlbo7z4Wp+k5R3bhhmshxYSYwY/LJdC9s5zt6THEs1OL1nATx41a9ZgezK12Z8RCAyX1V5P+MWFnus1qePyw0cAb5k4GDqA/bbwkUmKXQFo76sD8r8WEiirkUZSIMHj4M8owjfC6hYKM/3PA/+dr8HQJof2zXJqWacP/gZCnfQQiddyGkRiol+dAAy5Gn+gKUE90YAQ==', 'thinking': 'The user paid in USD and wants to know the amount in INR (Indian Rupees). The total in USD is $38.44.\n\nI need to:\n1. Get the exchange rate from USD to INR\n2. Convert $38.44 to INR\n\nL

<content-block:tool_call>
{
  "type": "tool_use",
  "id": "toolu_014ugAMgX8wjuxHm1T1ZthgX",
  "name": "multiply",
  "input": {
    "a": 38.44,
    "b": 83.1
  }
}
</content-block:tool_call>

INFO:anthropic_agent.src.core.retry:Anthropic stream completed successfully (stop_reason: tool_use)
DEBUG:anthropic_agent.src.core.agent:Assistant message: {'content': [{'id': 'toolu_014ugAMgX8wjuxHm1T1ZthgX', 'input': {'a': 38.44, 'b': 83.1}, 'name': 'multiply', 'type': 'tool_use'}], 'role': 'assistant'}
INFO:anthropic_agent.src.core.retry:Attempting Anthropic stream call (attempt 1/5)
DEBUG:anthropic_agent.src.core.retry:Anthropic request params: {'model': 'claude-sonnet-4-5', 'max_tokens': 64000, 'messages': [{'role': 'user', 'content': [{'type': 'image', 'source': {'type': 'base64', 'media_type': 'image/png', 'data': '...'}}, {'type': 'text', 'text': 'I paid the receipt in USD. What is the amount in INR?'}]}, {'content': [{'signature': 'EsQDCkYIChgCKkBeW1i5nXw+HOoIF7P/7KpNrly1Jyi6/dlo6xhKqwUcYBiT3smd3tnd6AmUXWr+8M4QNiDvBhK06M9REvh/79dfEgwRNMyCFIAO5lGLOBgaDO+zLwEMqo+b+nnFISIwLAAEPaJooG+ncv+KOls1ig3enE3su57+CKnstqMDozZaOUMyNTKcQ2BpQ3wbzT2gKqsCJwdKqcJ0qlqO5LzR3gHCYzUSlQFANzhm7FQuCY4/F

<content-block:text>The amount you paid in USD ($38.44) is approximately **₹3,194.36 INR** (Indian Rupees).

This is based on an exchange rate of 1 USD = 83.1 INR.
</content-block:text>

INFO:anthropic_agent.src.core.retry:Anthropic stream completed successfully (stop_reason: end_turn)
DEBUG:anthropic_agent.src.core.agent:Assistant message: {'content': [{'text': 'The amount you paid in USD ($38.44) is approximately **₹3,194.36 INR** (Indian Rupees).\n\nThis is based on an exchange rate of 1 USD = 83.1 INR.', 'type': 'text'}], 'role': 'assistant'}


In [18]:
result.__dict__

{'final_message': ParsedBetaMessage(id='msg_017bQjVey6MtPoKqdkfquUeD', container=None, content=[ParsedBetaTextBlock(citations=None, text='The amount you paid in USD ($38.44) is approximately **₹3,194.36 INR** (Indian Rupees).\n\nThis is based on an exchange rate of 1 USD = 83.1 INR.', type='text', parsed_output=None)], context_management=None, model='claude-sonnet-4-5-20250929', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=BetaUsage(cache_creation=BetaCacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=3814, output_tokens=56, server_tool_use=None, service_tier='standard')),
 'conversation_history': [{'role': 'user',
   'content': [{'type': 'image',
     'source': {'type': 'base64',
      'media_type': 'image/png',
      'data': '...'}},
    {'type': 'text',
     'text': 'I paid the receipt in USD. What is the amount in INR?'}]},
  {'content': [{'signature':

In [19]:
prompt_2 = {
    "role": "user",
    "content": [
        {
            "type": "text",
            "text": "Now generate a recepit image for the amount of various\
                items in INR only. Export and return the image to me."
        }
    ]
}

In [20]:
queue: asyncio.Queue[str | None] = asyncio.Queue()

async def print_queue() -> None:
        while True:
            chunk = await queue.get()
            if chunk is None:
                break
            print(chunk, end="", flush=True)
            queue.task_done()

printer = asyncio.create_task(print_queue())

result = await agent.run(prompt_2, queue, "xml")

await queue.put(None)  # signal printer to stop
await printer

DEBUG:anthropic_agent.src.core.agent:Anthropic count_tokens params: {'model': 'claude-sonnet-4-5', 'messages': [{'role': 'user', 'content': [{'type': 'image', 'source': {'type': 'base64', 'media_type': 'image/png', 'data': '...'}}, {'type': 'text', 'text': 'I paid the receipt in USD. What is the amount in INR?'}]}, {'content': [{'signature': 'EsQDCkYIChgCKkBeW1i5nXw+HOoIF7P/7KpNrly1Jyi6/dlo6xhKqwUcYBiT3smd3tnd6AmUXWr+8M4QNiDvBhK06M9REvh/79dfEgwRNMyCFIAO5lGLOBgaDO+zLwEMqo+b+nnFISIwLAAEPaJooG+ncv+KOls1ig3enE3su57+CKnstqMDozZaOUMyNTKcQ2BpQ3wbzT2gKqsCJwdKqcJ0qlqO5LzR3gHCYzUSlQFANzhm7FQuCY4/FJJmxdMRqEIBii9nWs8izRAuqyAyJspGBN3BLw/Ul0wiJPNuk2+sNOg6iQ1ZEoxhJk/uDlDr1zCLtDjeHNRYNIMtiLe29zM7cy2VprNQsevrJMcB8hsFrImeOK70smzFUUM0wfw0/Tn854UUiC7ewsfExnCDKHDwe+tVlbo7z4Wp+k5R3bhhmshxYSYwY/LJdC9s5zt6THEs1OL1nATx41a9ZgezK12Z8RCAyX1V5P+MWFnus1qePyw0cAb5k4GDqA/bbwkUmKXQFo76sD8r8WEiirkUZSIMHj4M8owjfC6hYKM/3PA/+dr8HQJof2zXJqWacP/gZCnfQQiddyGkRiol+dAAy5Gn+gKUE90YAQ==', 'thinking': 'The user paid in USD and wa

<content-block:thinking>The user wants me to create a receipt image similar to the original but with prices in INR only. I need to:
1. Convert all the USD prices to INR using the exchange rate
2. Create a receipt image with the INR prices
3. Export it to the OUTPUT_DIR

Let me first calculate all the prices in INR:
- Coffee Beans 1kg: $18.50 × 83.1 = 1,537.35 INR
- Green Tea 200g: $9.75 × 83.1 = 810.225 INR
- Chocolate Bar: $2.99 × 83.1 = 248.469 INR
- Imported Snacks: $7.20 × 83.1 = 598.32 INR
- Total: $38.44 × 83.1 = 3,194.364 INR

I'll use Python to create a receipt image using PIL or similar library, then export it.
</content-block:thinking><content-block:text>I'll create a receipt image with all prices in INR. Let me calculate the conversions and generate the image for you.
</content-block:text><content-block:tool_call>
{
  "type": "tool_use",
  "id": "toolu_013Yf36TPPo4wpb1zBfJJuvs",
  "name": "multiply",
  "input": {
    "a": 18.5,
    "b": 83.1
  }
}
</content-block:tool_call><

INFO:anthropic_agent.src.core.retry:Anthropic stream completed successfully (stop_reason: tool_use)
DEBUG:anthropic_agent.src.core.agent:Assistant message: {'content': [{'signature': 'EogGCkYIChgCKkDa8db6KHPBfPiOENfCEAfMTT+YRN16nZtYIu732lFmaK0RSsNxYnNV+geyAyndw9wtMj0jlksit9YfJ3N9mzAxEgzrsFZfKffUP4FdnqwaDAGeAIYvsB8ElOC4eCIwzwd3DK0F3AYhiWR+Oy1259jyMDG547D9OsS8K0zmDDMfiDdbnZP67X8vD5P0uD1UKu8Ei7xwcLA7CyoSGa4P3Z4tybF7CsWiQ4CReqa8TqRUCNwPq1HBloc/LCUlfPRyOKGGxuzjwC5GDj0E0FJCOgAWQXkC/86rQ4VFF2Xl+nLZMCBZF8TUFchoAetEdx7Sa0S/xANduRxS38WIxCZEJc9VvvuTSafk0fPnZa8gDRdpMAuuEakUlPclxyTuaaKER/DHU8ZM2oRfkY1Wj1jEc95cW0I6V8P03iai/zOwRdK0Uw8uUrY7H7ADq/Jyzq33/5d0WolUY4msmWHq/as6zqi5nVJ4xVdGxGdPEJoer79tjFT4ADecUVc57qgyYnsNDdBtKOlE3dYc/aQfwOUiCzTR7Dylbsn1HIj0UKccgcBZWeIVj7b3bxxWvWlc6Zd5Joy9YST7kjVe6iLdbGO9JOBfoW5dmpajWJcPyuqJ3zBtvYtDeS3hJtC6Q010klMQK/HAjpNM8hpPy+1sJDyA6i8SV1X3P8iqKlECT7s1CoELB71t3dGbPVRn7o/GX6kQ0NzkIsDDdSYMTPtQLs6tVAuujy7WH3uFwwtJnOUI7Bc0RctgWX2xu4O/uVpGjnhPkzVBQus9AlJuSb14vDD+KoZ5VVyru4mALvcM

<content-block:text>Now let me create the receipt image:
</content-block:text><content-block:server_tool_call>
{
  "type": "server_tool_use",
  "id": "srvtoolu_014FDG9hM2raKUS3vscBzbim",
  "name": "text_editor_code_execution",
  "input": {
    "command": "create",
    "path": "/tmp/create_receipt.py",
    "file_text": "from PIL import Image, ImageDraw, ImageFont\nimport os\n\n# Create a new image with white background\nwidth = 600\nheight = 700\nimage = Image.new('RGB', (width, height), 'white')\ndraw = ImageDraw.Draw(image)\n\n# Try to use a default font, with fallback\ntry:\n    title_font = ImageFont.truetype(\"/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf\", 40)\n    header_font = ImageFont.truetype(\"/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf\", 28)\n    normal_font = ImageFont.truetype(\"/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf\", 20)\n    small_font = ImageFont.truetype(\"/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf\", 16)\nexcept:\n    title_font = Im

INFO:anthropic_agent.src.core.retry:Anthropic stream completed successfully (stop_reason: end_turn)
DEBUG:anthropic_agent.src.core.agent:Assistant message: {'content': [{'text': 'Now let me create the receipt image:', 'type': 'text'}, {'id': 'srvtoolu_014FDG9hM2raKUS3vscBzbim', 'input': {'command': 'create', 'path': '/tmp/create_receipt.py', 'file_text': 'from PIL import Image, ImageDraw, ImageFont\nimport os\n\n# Create a new image with white background\nwidth = 600\nheight = 700\nimage = Image.new(\'RGB\', (width, height), \'white\')\ndraw = ImageDraw.Draw(image)\n\n# Try to use a default font, with fallback\ntry:\n    title_font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 40)\n    header_font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 28)\n    normal_font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 20)\n    small_font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 


<content-block:meta_files>
{
  "file_ids": [
    "file_011CVXx55SD6M5Nx1sxahGjT"
  ]
}
</content-block:meta_files>

In [21]:
result.__dict__

{'final_message': ParsedBetaMessage(id='msg_012RphWsPPZo5gaaWKkjC8U6', container=BetaContainer(id='container_011CVXx2yRVN4g6q3dpWUDpt', expires_at=datetime.datetime(2025, 11, 27, 7, 50, 49, 248459, tzinfo=TzInfo(0)), skills=None), content=[ParsedBetaTextBlock(citations=None, text='Now let me create the receipt image:', type='text', parsed_output=None), BetaServerToolUseBlock(id='srvtoolu_014FDG9hM2raKUS3vscBzbim', caller=None, input={'command': 'create', 'path': '/tmp/create_receipt.py', 'file_text': 'from PIL import Image, ImageDraw, ImageFont\nimport os\n\n# Create a new image with white background\nwidth = 600\nheight = 700\nimage = Image.new(\'RGB\', (width, height), \'white\')\ndraw = ImageDraw.Draw(image)\n\n# Try to use a default font, with fallback\ntry:\n    title_font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 40)\n    header_font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 28)\n    normal_font = ImageFont.tru

In [22]:
agent.messages

[{'role': 'user',
  'content': [{'type': 'image',
    'source': {'type': 'base64',
     'media_type': 'image/png',
     'data': '...'}},
   {'type': 'text',
    'text': 'I paid the receipt in USD. What is the amount in INR?'}]},
 {'content': [{'signature': 'EsQDCkYIChgCKkBeW1i5nXw+HOoIF7P/7KpNrly1Jyi6/dlo6xhKqwUcYBiT3smd3tnd6AmUXWr+8M4QNiDvBhK06M9REvh/79dfEgwRNMyCFIAO5lGLOBgaDO+zLwEMqo+b+nnFISIwLAAEPaJooG+ncv+KOls1ig3enE3su57+CKnstqMDozZaOUMyNTKcQ2BpQ3wbzT2gKqsCJwdKqcJ0qlqO5LzR3gHCYzUSlQFANzhm7FQuCY4/FJJmxdMRqEIBii9nWs8izRAuqyAyJspGBN3BLw/Ul0wiJPNuk2+sNOg6iQ1ZEoxhJk/uDlDr1zCLtDjeHNRYNIMtiLe29zM7cy2VprNQsevrJMcB8hsFrImeOK70smzFUUM0wfw0/Tn854UUiC7ewsfExnCDKHDwe+tVlbo7z4Wp+k5R3bhhmshxYSYwY/LJdC9s5zt6THEs1OL1nATx41a9ZgezK12Z8RCAyX1V5P+MWFnus1qePyw0cAb5k4GDqA/bbwkUmKXQFo76sD8r8WEiirkUZSIMHj4M8owjfC6hYKM/3PA/+dr8HQJof2zXJqWacP/gZCnfQQiddyGkRiol+dAAy5Gn+gKUE90YAQ==',
    'thinking': 'The user paid in USD and wants to know the amount in INR (Indian Rupees). The total in USD is $38.44.\n\nI nee

In [ ]:
await agent.drain_background_tasks()

{'total_tasks': 0, 'completed': 0, 'timed_out': 0, 'task_ids': []}